<h1 class="">Feature Engineering</h1>
<h5 class="">Richter's Predictor: Modeling Earthquake Damage</h5>
<h5 class="">Competition hosted by <a href="https://www.drivendata.org/competitions/57/nepal-earthquake/page/134/">Driven Data</a></h5>
<h5 class=""><a href="https://github.com/hss0727/driven_data_richters_predictor">Github Repo for code</a></h5>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt

# importing machine learning models for prediction
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.svm import SVC, LinearSVC

#PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale 
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_squared_error
from sklearn import model_selection

pd.options.display.max_columns = 999

In [2]:
# Read data
train_values = pd.read_csv('data/train_values.csv')
train_labels = pd.read_csv('data/train_labels.csv')
test_values = pd.read_csv('data/test_values.csv')

In [3]:
train_values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 39 columns):
 #   Column                                  Non-Null Count   Dtype 
---  ------                                  --------------   ----- 
 0   building_id                             260601 non-null  int64 
 1   geo_level_1_id                          260601 non-null  int64 
 2   geo_level_2_id                          260601 non-null  int64 
 3   geo_level_3_id                          260601 non-null  int64 
 4   count_floors_pre_eq                     260601 non-null  int64 
 5   age                                     260601 non-null  int64 
 6   area_percentage                         260601 non-null  int64 
 7   height_percentage                       260601 non-null  int64 
 8   land_surface_condition                  260601 non-null  object
 9   foundation_type                         260601 non-null  object
 10  roof_type                               260601 non-null 

In [4]:
# PCA
X = train_values.iloc[:,15:25]
X = pd.get_dummies(X, prefix_sep='_')

pca = PCA()
principalComponents = pca.fit_transform(scale(X))

PCA_df = pd.DataFrame(data=principalComponents[:,:5], columns=['PC1','PC2','PC3','PC4','PC5'])

# Add PCA components to DF
new_train_values = pd.concat([train_values, PCA_df], axis=1)

# Drop columns used for PCA
# columns #15 ~ #25: "count_floors_pre_eq" ~ "plan_configuration"
drop_list = new_train_values.columns[15:26]
new_train_values.drop(drop_list, axis=1, inplace=True)

In [5]:
PCA_df

,PC1,PC2,PC3,PC4,PC5
0,0.408014,-0.088451,-2.021006,0.384819,0.277794
1,-0.786669,-0.645932,-0.180148,0.027844,-0.022930
2,-0.786669,-0.645932,-0.180148,0.027844,-0.022930
3,-0.794170,3.102991,0.866614,-0.916307,-0.913301
4,1.941554,0.024824,-1.853545,0.243018,0.274190
...,...,...,...,...,...
260596,-0.786669,-0.645932,-0.180148,0.027844,-0.022930
260597,-0.786669,-0.645932,-0.180148,0.027844,-0.022930
260598,-0.786669,-0.645932,-0.180148,0.027844,-0.022930
260599,2.375798,-1.212608,1.391386,-0.522210,-0.576310


In [6]:
new_train_df = pd.merge(new_train_values, train_labels, on="building_id")

In [11]:
### Create Conditional Probability Table ###

In [12]:
# Count number of each geographical ID
# geo1_count: number of geo_id_1 (from 1 to 30)
# geo2_count: number of geo_id_2 (from 1 to 1,427)
# geo3_count: number of geo_id_2 (from 1 to 12,567)
geo1_count = pd.DataFrame(new_train_df['geo_level_1_id'].value_counts().sort_index())
geo2_count = pd.DataFrame(new_train_df['geo_level_2_id'].value_counts().sort_index())
geo3_count = pd.DataFrame(new_train_df['geo_level_3_id'].value_counts().sort_index())

geo1_count.rename(columns={"geo_level_1_id":"counts"}, inplace=True)
geo2_count.rename(columns={"geo_level_2_id":"counts"}, inplace=True)
geo3_count.rename(columns={"geo_level_3_id":"counts"}, inplace=True)

In [13]:
geo1_count.head()

,counts
0,4011
1,2701
2,931
3,7540
4,14568


In [14]:
# Fill N/A with interpolated value
index_geo1 = list(range(31))
a = pd.DataFrame()
a['id'] = index_geo1
geo1_count=pd.merge(a, geo1_count, how='left', left_on='id', right_index=True)
geo1_count=geo1_count.interpolate()

In [27]:
### Calculate conditional probability of geo_id_1 ==1 and damage_grade = 1,2,3
# Find the number of observation in the intersection of geo_id and damage grade
dam1 = []

for i in range(0,31):
    dam1_count = len(new_train_df[new_train_df.geo_level_1_id==i][new_train_df.damage_grade==1])
    dam1.append(dam1_count)

geo1_count["damage_1"] = pd.DataFrame(dam1)

dam2 = []

for i in range(0,31):
    dam2_count = len(new_train_df[new_train_df.geo_level_1_id==i][new_train_df.damage_grade==2])
    dam2.append(dam2_count)

geo1_count["damage_2"] = pd.DataFrame(dam2)

dam3 = []

for i in range(0,31):
    dam3_count = len(new_train_df[new_train_df.geo_level_1_id==i][new_train_df.damage_grade==3])
    dam3.append(dam3_count)

geo1_count["damage_3"] = pd.DataFrame(dam3)

# Calculate conditional probability
geo1_count['geo1_dam1_prob'] = geo1_count['damage_1'] / geo1_count['counts']
geo1_count['geo1_dam2_prob'] = geo1_count['damage_2'] / geo1_count['counts']
geo1_count['geo1_dam3_prob'] = geo1_count['damage_3'] / geo1_count['counts']

/var/folders/gm/kys_b_lx5n153wpkc5l01s280000gn/T/ipykernel_26128/2399922934.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dam1_count = len(new_train_df[new_train_df.geo_level_1_id==i][new_train_df.damage_grade==1])
/var/folders/gm/kys_b_lx5n153wpkc5l01s280000gn/T/ipykernel_26128/2399922934.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dam2_count = len(new_train_df[new_train_df.geo_level_1_id==i][new_train_df.damage_grade==2])
/var/folders/gm/kys_b_lx5n153wpkc5l01s280000gn/T/ipykernel_26128/2399922934.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dam3_count = len(new_train_df[new_train_df.geo_level_1_id==i][new_train_df.damage_grade==3])


In [15]:
# Fill N/A with interpolated value
index_geo2 = list(range(1428))
a = pd.DataFrame()
a['id'] = index_geo2
geo2_count=pd.merge(a, geo2_count, how='left', left_on='id', right_index=True)
geo2_count=geo2_count.interpolate()
#geo2_count.drop("id", axis=1, inplace=True)


In [16]:
### Calculate conditional probability of geo_id_1 == 2 and damage_grade = 1,2,3
# Find the number of observation in the intersection of geo_id and damage grade
dam1 = []

for i in range(0,1428):
    dam1_count = len(new_train_df[new_train_df.geo_level_2_id==i][new_train_df.damage_grade==1])
    dam1.append(dam1_count)

geo2_count["damage_1"] = pd.DataFrame(dam1)

dam2 = []

for i in range(0,1428):
    dam2_count = len(new_train_df[new_train_df.geo_level_2_id==i][new_train_df.damage_grade==2])
    dam2.append(dam2_count)

geo2_count["damage_2"] = pd.DataFrame(dam2)

dam3 = []

for i in range(0,1428):
    dam3_count = len(new_train_df[new_train_df.geo_level_2_id==i][new_train_df.damage_grade==3])
    dam3.append(dam3_count)

geo2_count["damage_3"] = pd.DataFrame(dam3)
geo2_count=geo2_count.interpolate()

# Calculate conditional probability
geo2_count['geo2_dam1_prob'] = geo2_count['damage_1'] / geo2_count['counts']
geo2_count['geo2_dam2_prob'] = geo2_count['damage_2'] / geo2_count['counts']
geo2_count['geo2_dam3_prob'] = geo2_count['damage_3'] / geo2_count['counts']

/var/folders/gm/kys_b_lx5n153wpkc5l01s280000gn/T/ipykernel_26128/1145853777.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dam1_count = len(new_train_df[new_train_df.geo_level_2_id==i][new_train_df.damage_grade==1])
/var/folders/gm/kys_b_lx5n153wpkc5l01s280000gn/T/ipykernel_26128/1145853777.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dam2_count = len(new_train_df[new_train_df.geo_level_2_id==i][new_train_df.damage_grade==2])
/var/folders/gm/kys_b_lx5n153wpkc5l01s280000gn/T/ipykernel_26128/1145853777.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dam3_count = len(new_train_df[new_train_df.geo_level_2_id==i][new_train_df.damage_grade==3])


In [17]:
# Fill N/A with interpolated value
index_geo3 = list(range(12568))
b = pd.DataFrame()
b['id'] = index_geo3
geo3_count=pd.merge(b, geo3_count, how='left', left_on='id', right_index=True)
geo3_count=geo3_count.interpolate()
##geo3_count.drop("id", axis=1, inplace=True)

In [18]:
### Calculate conditional probability of geo_id_1 == 3 and damage_grade = 1,2,3
# Find the number of observation in the intersection of geo_id and damage grade
dam1 = []

for i in range(0,12568):
    dam1_count = len(new_train_df[new_train_df.geo_level_3_id==i][new_train_df.damage_grade==1])
    dam1.append(dam1_count)

geo3_count["damage_1"] = pd.DataFrame(dam1)

dam2 = []

for i in range(0,12568):
    dam2_count = len(new_train_df[new_train_df.geo_level_3_id==i][new_train_df.damage_grade==2])
    dam2.append(dam2_count)

geo3_count["damage_2"] = pd.DataFrame(dam2)

dam3 = []

for i in range(0,12568):
    dam3_count = len(new_train_df[new_train_df.geo_level_3_id==i][new_train_df.damage_grade==3])
    dam3.append(dam3_count)

geo3_count["damage_3"] = pd.DataFrame(dam3)
geo3_count=geo3_count.interpolate()

# Calculate conditional probability
geo3_count['geo3_dam1_prob'] = geo3_count['damage_1'] / geo3_count['counts']
geo3_count['geo3_dam2_prob'] = geo3_count['damage_2'] / geo3_count['counts']
geo3_count['geo3_dam3_prob'] = geo3_count['damage_3'] / geo3_count['counts']

/var/folders/gm/kys_b_lx5n153wpkc5l01s280000gn/T/ipykernel_26128/2776618492.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dam1_count = len(new_train_df[new_train_df.geo_level_3_id==i][new_train_df.damage_grade==1])
/var/folders/gm/kys_b_lx5n153wpkc5l01s280000gn/T/ipykernel_26128/2776618492.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dam2_count = len(new_train_df[new_train_df.geo_level_3_id==i][new_train_df.damage_grade==2])
/var/folders/gm/kys_b_lx5n153wpkc5l01s280000gn/T/ipykernel_26128/2776618492.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dam3_count = len(new_train_df[new_train_df.geo_level_3_id==i][new_train_df.damage_grade==3])


In [28]:
geo1_count.head()

,id,counts,damage_1,damage_2,damage_3,geo1_dam1_prob,geo1_dam2_prob,geo1_dam3_prob
0,0,4011,337,3075,599,0.084019,0.766642,0.149339
1,1,2701,411,1985,305,0.152166,0.734913,0.112921
2,2,931,85,610,236,0.091300,0.655209,0.253491
3,3,7540,245,4550,2745,0.032493,0.603448,0.364058
4,4,14568,521,11164,2883,0.035763,0.766337,0.197900


In [26]:
geo2_count.head()

,id,counts,damage_1,damage_2,damage_3,geo2_dam1_prob,geo2_dam2_prob,geo2_dam3_prob
0,0,38.0,0,9,29,0.000000,0.236842,0.763158
1,1,204.0,0,133,71,0.000000,0.651961,0.348039
2,2,140.5,0,0,0,0.000000,0.000000,0.000000
3,3,77.0,5,59,13,0.064935,0.766234,0.168831
4,4,315.0,2,147,166,0.006349,0.466667,0.526984


In [29]:
geo3_count.head()

,id,counts,damage_1,damage_2,damage_3,geo3_dam1_prob,geo3_dam2_prob,geo3_dam3_prob
0,0,2.0,0,0,2,0.000000,0.000000,1.000000
1,1,6.0,1,1,4,0.166667,0.166667,0.666667
2,2,7.5,0,0,0,0.000000,0.000000,0.000000
3,3,9.0,1,8,0,0.111111,0.888889,0.000000
4,4,11.5,0,0,0,0.000000,0.000000,0.000000


In [30]:
# Extract and create a table with geo_id == 1,2,3 to create a conditional probability table
prob_table = new_train_df.iloc[:,0:4]


In [31]:
prob_table.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id
0,802906,6,487,12198
1,28830,8,900,2812
2,94947,21,363,8973
3,590882,22,418,10694
4,201944,11,131,1488


In [32]:
# Merge conditional probability tables
temp = geo1_count.iloc[:,[0,5,6,7]]
prob_table = pd.merge(prob_table, temp, how="left", left_on="geo_level_1_id", right_on="id")
prob_table.drop("id", axis=1, inplace=True)

temp2 = geo2_count.iloc[:,[0,5,6,7]]
prob_table = pd.merge(prob_table, temp2, how="left", left_on="geo_level_2_id", right_on="id")
prob_table.drop("id", axis=1, inplace=True)

temp3 = geo3_count.iloc[:,[0,5,6,7]]
prob_table = pd.merge(prob_table, temp3, how="left", left_on="geo_level_3_id", right_on="id")
prob_table.drop("id", axis=1, inplace=True)

prob_table.drop(["geo_level_1_id", "geo_level_2_id", "geo_level_3_id"], axis=1, inplace=True)

In [35]:
prob_table.head()

,building_id,geo1_dam1_prob,geo1_dam2_prob,geo1_dam3_prob,geo2_dam1_prob,geo2_dam2_prob,geo2_dam3_prob,geo3_dam1_prob,geo3_dam2_prob,geo3_dam3_prob
0,802906,0.086461,0.665354,0.248185,0.003704,0.251852,0.744444,0.000000,0.162162,0.837838
1,28830,0.034277,0.446174,0.519549,0.010050,0.492462,0.497487,0.062500,0.812500,0.125000
2,94947,0.021627,0.393378,0.584996,0.082386,0.316477,0.601136,0.029412,0.360294,0.610294
3,590882,0.129718,0.739603,0.130678,0.019512,0.853659,0.126829,0.032258,0.838710,0.129032
4,201944,0.046959,0.568370,0.384672,0.029865,0.591522,0.378613,0.008197,0.614754,0.377049


In [36]:
# Merge the conditional probability table to the original dataset
new_train_df = pd.merge(new_train_df, prob_table, how="left", left_on="building_id", right_on="building_id")

In [37]:
# Drop unnecessary variables
drop_list2 = new_train_df.columns[14:28]
new_train_df.drop(drop_list2, axis=1, inplace=True)

In [38]:
new_train_df.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,PC1,PC2,PC3,PC4,PC5,damage_grade,geo1_dam1_prob,geo1_dam2_prob,geo1_dam3_prob,geo2_dam1_prob,geo2_dam2_prob,geo2_dam3_prob,geo3_dam1_prob,geo3_dam2_prob,geo3_dam3_prob
0,802906,6,487,12198,2,30,6,5,t,r,n,f,q,t,0.408014,-0.088451,-2.021006,0.384819,0.277794,3,0.086461,0.665354,0.248185,0.003704,0.251852,0.744444,0.000000,0.162162,0.837838
1,28830,8,900,2812,2,10,8,7,o,r,n,x,q,s,-0.786669,-0.645932,-0.180148,0.027844,-0.022930,2,0.034277,0.446174,0.519549,0.010050,0.492462,0.497487,0.062500,0.812500,0.125000
2,94947,21,363,8973,2,10,5,5,t,r,n,f,x,t,-0.786669,-0.645932,-0.180148,0.027844,-0.022930,3,0.021627,0.393378,0.584996,0.082386,0.316477,0.601136,0.029412,0.360294,0.610294
3,590882,22,418,10694,2,10,6,5,t,r,n,f,x,s,-0.794170,3.102991,0.866614,-0.916307,-0.913301,2,0.129718,0.739603,0.130678,0.019512,0.853659,0.126829,0.032258,0.838710,0.129032
4,201944,11,131,1488,3,30,8,9,t,r,n,f,x,s,1.941554,0.024824,-1.853545,0.243018,0.274190,3,0.046959,0.568370,0.384672,0.029865,0.591522,0.378613,0.008197,0.614754,0.377049


In [39]:
# One hot encode dummy variables
new_train_df = pd.get_dummies(new_train_df, prefix_sep='_')

In [40]:
# Extract labels from the train dataset
new_train_labels = new_train_df['damage_grade']
new_train_df.drop(['building_id','damage_grade','geo_level_1_id','geo_level_2_id','geo_level_3_id'], axis=1, inplace=True)

In [41]:
############################
### Clean Test Values
############################

### Same procedure with train dataset

In [42]:
# PCA
X2 = test_values.iloc[:,15:25]
X2 = pd.get_dummies(X2, prefix_sep='_')

pca_test = PCA()
principalComponents_test = pca_test.fit_transform(scale(X2))

PCA_df_test = pd.DataFrame(data=principalComponents_test[:,:5], columns=['PC1','PC2','PC3','PC4','PC5'])

test_values = pd.concat([test_values, PCA_df_test], axis=1)

# Drop columns used for PCA
# columns #15 ~ #25: "count_floors_pre_eq" ~ "plan_configuration"
drop_list2 = test_values.columns[15:26]
test_values.drop(drop_list2, axis=1, inplace=True)

In [45]:
prob_table2 = test_values.iloc[:,0:4]

In [46]:
temp = geo1_count.iloc[:,[0,5,6,7]]
prob_table2 = pd.merge(prob_table2, temp, how="left", left_on="geo_level_1_id", right_on="id")
prob_table2.drop("id", axis=1, inplace=True)

temp2 = geo2_count.iloc[:,[0,5,6,7]]
prob_table2 = pd.merge(prob_table2, temp2, how="left", left_on="geo_level_2_id", right_on="id")
prob_table2.drop("id", axis=1, inplace=True)

temp3 = geo3_count.iloc[:,[0,5,6,7]]
prob_table2 = pd.merge(prob_table2, temp3, how="left", left_on="geo_level_3_id", right_on="id")
prob_table2.drop("id", axis=1, inplace=True)

prob_table2.drop(["geo_level_1_id", "geo_level_2_id", "geo_level_3_id"], axis=1, inplace=True)

In [47]:
test_values = pd.merge(test_values, prob_table2, how="left", left_on="building_id", right_on="building_id")

In [48]:
drop_list3 = test_values.columns[14:28]
test_values.drop(drop_list3, axis=1, inplace=True)

In [49]:
test_values = pd.get_dummies(test_values, prefix_sep='_')
test_values.drop(['building_id','geo_level_1_id','geo_level_2_id','geo_level_3_id'], axis=1, inplace=True)

In [50]:
# Save train and test dataset
new_train_df.to_csv('V3_train_feat_engineered.csv')
new_train_labels.to_csv('V3_train_labels_feat_engineered.csv')
test_values.to_csv('V3_test_feat_engineered.csv')